In [16]:
from os.path import dirname, join
from pprint import pprint

import numpy as np
import ipywidgets as ipyw
import matplotlib.pyplot as plt
import pydicom
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir

class ImageSliceViewer3D:
    """ 
    ImageSliceViewer3D is for viewing volumetric image slices in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image and 
    the slice plane being viewed. 

    Argumentss:
    Volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('gray'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/users/colormaps.html
    
    """
    
    def __init__(self, volume, figsize=(8,8), cmap='gray'):
        self.volume = volume
        self.figsize = figsize
        self.cmap = cmap
        self.v = [np.min(volume), np.max(volume)]
        
        # Call to select slice plane
        ipyw.interact(self.view_selection, view=ipyw.RadioButtons(
            options=['x-y','y-z', 'z-x'], value='x-y', 
            description='Slice plane selection:', disabled=False,
            style={'description_width': 'initial'}))
    
    def view_selection(self, view):
        # Transpose the volume to orient according to the slice plane selection
        orient = {"y-z":[1,2,0], "z-x":[2,0,1], "x-y": [0,1,2]}
        self.vol = np.transpose(self.volume, orient[view])
        maxZ = self.vol.shape[2] - 1
        
        # Call to view a slice within the selected slice plane
        ipyw.interact(self.plot_slice, 
            z=ipyw.IntSlider(min=0, max=maxZ, step=1, continuous_update=False, 
            description='Image Slice:'))
        
    def plot_slice(self, z):
        # Plot slice for the given plane and slice
        self.fig = plt.figure(figsize=self.figsize)
        plt.imshow(self.vol[:,:,z], cmap=plt.get_cmap(self.cmap), 
            vmin=self.v[0], vmax=self.v[1])

        
# fetch the path to the test data
filepath = 'C:\\Users\\eramirez\\Desktop\\CT\\LENS_P3\\INSP_CPAP\\DICOMDIR'
print('Path to the DICOM directory: {}'.format(filepath))

# load the data
dicom_dir = read_dicomdir(filepath)
base_dir = dirname(filepath)

patient_record = dicom_dir.patient_records[0] #select one patient

if (hasattr(patient_record, 'PatientID') and
            hasattr(patient_record, 'PatientName')):
        print("Patient: {}: {}".format(patient_record.PatientID,
                                       patient_record.PatientName))
study = patient_record.children[0]
#print(study)
print(" " * 4 + "Study {}: {}: {}".format(study.StudyID,
                                                  study.StudyDate,
                                                  study.StudyDescription))
series = study.children[0]
image_records = series.children

image_filenames = [join(base_dir, *image_rec.ReferencedFileID)
                               for image_rec in image_records]

#get the pixel array
datasets = [pydicom.dcmread(image_filename).pixel_array
                        for image_filename in image_filenames]
#convert to numpy array
npa = np.array(datasets)
ImageSliceViewer3D(npa)


Path to the DICOM directory: C:\Users\eramirez\Desktop\CT\LENS_P3\INSP_CPAP\DICOMDIR
Patient: 12995477: CIVICO CASADO^PONCIANO
    Study 7951: 20150716: Tc de torax


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…